In [1]:
# 2022/01/05 업데이트 내용:
# 건물연식 2년이하 자료들도 생성

In [2]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
import datetime
tqdm.tqdm.pandas()

In [3]:
filename = os.listdir('../')[2]

In [4]:
target_df = pd.read_excel('../' + filename)
target_df['계약날짜'] = pd.to_datetime(target_df['계약날짜'])
target_df = target_df.drop(columns=['Unnamed: 16', 'Unnamed: 17'])
target_df['지번주소_건축년도'] = target_df['지번주소'] + ' ' + target_df['건축년도'].astype('str')
print(target_df.shape)
target_df.head()

(864, 17)


,지번주소,건물(단지)명,건축년도,계약날짜,부동산유형,소유자,평균 전용면적(㎡),총 전용면적(㎡),전용면적 표준편차(㎡),평균 거래금액(만원),총 거래금액(만원),거래금액 표준편차(만원),최대 층,평균 단가(만원),단가 표준편차(만원),세대 수,지번주소_건축년도
0,강남구 개포동 1237-7,한별2,2021,2021-06-01,오피스텔,SH,29.665000,118.66,0.390000,33565.250000,134261,437.500000,2,1131.477759,0.128995,4,강남구 개포동 1237-7 2021
1,강남구 개포동 1237-7,한별2,2021,2021-06-01,연립다세대,SH,29.946667,449.20,0.037544,38163.333333,572450,44.185755,6,1274.376854,0.358219,15,강남구 개포동 1237-7 2021
2,강남구 개포동 1216-7,백년빌,2020,2020-03-18,연립다세대,SH,26.686000,400.29,2.367074,30573.333333,458600,2134.651750,5,1147.330195,25.484846,15,강남구 개포동 1216-7 2020
3,강남구 개포동 1195-10,동영빌,2020,2020-03-02,연립다세대,SH,28.814667,432.22,4.855684,31463.333333,471950,4139.637090,5,1097.178201,36.299709,15,강남구 개포동 1195-10 2020
4,강남구 개포동 1199-7,개포백년빌,2019,2019-12-19,연립다세대,LH,27.905000,390.67,3.070186,28857.142857,404000,3536.722578,5,1033.367584,37.609677,14,강남구 개포동 1199-7 2019


In [5]:
def day_modifier(x):
    # x is a number
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x)
        if len(x) == 1:
            return '0' + x
        else:
            return x

In [6]:
def landnum_modifier(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x).replace('외', '').replace(' ','')
        splitted = x.split('-')
        if len(splitted) == 1:
            return x + '-0'
        else:
            return x

In [7]:
def data_prep(bdtype, tradetype, starting_year=2018):
    # bdtype is a string: one of 아파트, 연립다세대 or 오피스텔
    # tradetype is a string: one of 매매 or 전월세
    if (tradetype != '매매') & (tradetype != '전월세'):
        raise ValueError('두번째 변수는 매매 또는 전월세만 입력 가능')
    
    
    basedir = './국토교통부_실거래가_공개시스템/{}/{}/'.format(bdtype, tradetype)
    filenames = [f for f in os.listdir(basedir) if (f.endswith('.csv'))&('(' in f)]
    
    dfs_list = []
    for i, f in tqdm.tqdm_notebook(enumerate(filenames)):
        splitted_filename = f.split('실거래가_')       
        file_year = int(splitted_filename[-1][:4])
        
        '''if file_year < starting_year:
            continue'''
        
        try:
            df = pd.read_csv(basedir + f, encoding='euc-kr', header=15)
        except:
            try:
                df = pd.read_csv(basedir + f, encoding='utf-8', header=15)
            except:
                df = pd.read_csv(basedir + f, header=15)
        
        if '건물명' in df.columns:
            df.rename(columns={'건물명':'건물(단지)명'}, inplace=True)
        elif '단지명' in df.columns:
            df.rename(columns={'단지명':'건물(단지)명'}, inplace=True)
            
        if '대지권면적(㎡)' in df.columns:
            df = df.drop(columns=['대지권면적(㎡)'])
                
            
        if '해제사유발생일' in df.columns.tolist():
            df = df[df['해제사유발생일'].isna()]
            df = df.drop(columns=['해제사유발생일'])
            
        dfs_list.append(df)
    
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    if tradetype == '전월세':
        concat_df = concat_df.rename(columns={'전월세구분':'거래구분', '보증금(만원)':'거래금액(만원)'})
        concat_df = concat_df.drop(columns=['월세(만원)'])
        concat_df = concat_df[concat_df['거래구분'] == '전세']
    elif tradetype == '매매':
        concat_df['거래구분'] = '매매'
        
    concat_df['번지'] = concat_df['번지'].apply(landnum_modifier)
    
    concat_df['계약년월'] = concat_df['계약년월'].astype('Int64')
    concat_df['계약일'] = concat_df['계약일'].astype('Int64')
    
    concat_df['계약년월'] = concat_df['계약년월'].apply(str)
    concat_df['계약일'] = concat_df['계약일'].apply(str)
    
    concat_df['계약날짜기준_건물연식'] = concat_df['계약년월'].apply(lambda x: int(x[:4])) - concat_df['건축년도']
    
    concat_df['계약일'] = concat_df['계약일'].apply(day_modifier)
    
    concat_df['계약날짜'] = concat_df['계약년월'].apply(lambda x: x[:4]) + '-' + concat_df['계약년월'].apply(lambda x: x[-2:])\
    + '-' + concat_df['계약일']
    
    concat_df['계약날짜'] = pd.to_datetime(concat_df['계약날짜'], format='%Y-%m-%d')
    
    concat_df['거래금액(만원)'] = concat_df['거래금액(만원)'].apply(lambda x: int(x.replace(',','')))
    concat_df['단가(만원/㎡)'] = concat_df['거래금액(만원)'] / concat_df['전용면적(㎡)']
    
    concat_df['지번주소'] = concat_df['시군구'] + ' ' + concat_df['번지']
        
    cols_to_drop = ['번지', '지번주소', '계약년월', '계약일', '도로명']
        
    concat_df = concat_df[['지번주소', '도로명'] + [col for col in concat_df.columns if col not in cols_to_drop]]
    
    date_today = pd.to_datetime(datetime.datetime.now().strftime('%Y-%m-%d'), format='%Y-%m-%d')
    concat_df['건물연식'] = date_today.year - concat_df['건축년도']
    
    concat_df = concat_df.dropna(subset=['지번주소'])
    
    concat_df = concat_df[concat_df['층'] >= 0].reset_index(drop=True)
    
    return concat_df

In [8]:
apart_trade_df = data_prep('아파트', '매매')
apart_lease_df = data_prep('아파트', '전월세')
apart_df = pd.concat([apart_trade_df, apart_lease_df]).reset_index(drop=True)
del apart_trade_df, apart_lease_df
apart_df['부동산유형'] = '아파트'
print(apart_df.shape)
apart_df.head()

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


0it [00:00, ?it/s]

0it [00:00, ?it/s]

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:2: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  
c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:2: DtypeWarning: Columns (10,18) have mixed types.Specify dtype option on import or set low_memory=False.
  


(2526354, 23)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,거래유형,중개사소재지,거래구분,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,계약기간,계약구분,갱신요구권 사용,종전계약 보증금 (만원),종전계약 월세 (만원),부동산유형
0,서울특별시 강남구 개포동 655-2,언주로 103,서울특별시 강남구 개포동,655.0,2.0,개포2차현대아파트(220),77.75,59500,7.0,1988.0,-,-,매매,18.0,2006-03-10,765.273312,34.0,NaN,NaN,NaN,NaN,NaN,아파트
1,서울특별시 강남구 개포동 655-2,언주로 103,서울특별시 강남구 개포동,655.0,2.0,개포2차현대아파트(220),77.75,60000,6.0,1988.0,-,-,매매,18.0,2006-03-29,771.704180,34.0,NaN,NaN,NaN,NaN,NaN,아파트
2,서울특별시 강남구 개포동 655-2,언주로 103,서울특별시 강남구 개포동,655.0,2.0,개포2차현대아파트(220),77.75,67000,9.0,1988.0,-,-,매매,18.0,2006-04-29,861.736334,34.0,NaN,NaN,NaN,NaN,NaN,아파트
3,서울특별시 강남구 개포동 655-2,언주로 103,서울특별시 강남구 개포동,655.0,2.0,개포2차현대아파트(220),77.75,60000,4.0,1988.0,-,-,매매,18.0,2006-06-01,771.704180,34.0,NaN,NaN,NaN,NaN,NaN,아파트
4,서울특별시 강남구 개포동 655-2,언주로 103,서울특별시 강남구 개포동,655.0,2.0,개포2차현대아파트(220),77.75,72250,5.0,1988.0,-,-,매매,18.0,2006-10-20,929.260450,34.0,NaN,NaN,NaN,NaN,NaN,아파트


In [9]:
yunrip_trade_df = data_prep('연립다세대', '매매')
yunrip_lease_df = data_prep('연립다세대', '전월세')
yunrip_df = pd.concat([yunrip_trade_df, yunrip_lease_df]).reset_index(drop=True)
del yunrip_trade_df, yunrip_lease_df
yunrip_df['부동산유형'] = '연립다세대'
print(yunrip_df.shape)
yunrip_df.head()

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


0it [00:00, ?it/s]

0it [00:00, ?it/s]

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:2: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  


(1274760, 23)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,거래유형,중개사소재지,거래구분,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,계약기간,계약구분,갱신요구권 사용,종전계약 보증금 (만원),종전계약 월세 (만원),부동산유형
0,서울특별시 강남구 개포동 171-13,선릉로14길 11,서울특별시 강남구 개포동,171,13,(171-13),68.08,56500,2.0,1988.0,-,-,매매,18.0,2006-12-20,829.905993,34.0,NaN,NaN,NaN,NaN,NaN,연립다세대
1,서울특별시 강남구 개포동 1239-7,개포로15길 25,서울특별시 강남구 개포동,1239,7,강남빌라가동,52.59,40500,2.0,1988.0,-,-,매매,18.0,2006-12-09,770.108386,34.0,NaN,NaN,NaN,NaN,NaN,연립다세대
2,서울특별시 강남구 개포동 1239-7,개포로15길 25,서울특별시 강남구 개포동,1239,7,강남빌라가동,52.59,42000,1.0,1988.0,-,-,매매,18.0,2006-12-19,798.630918,34.0,NaN,NaN,NaN,NaN,NaN,연립다세대
3,서울특별시 강남구 개포동 1239-6,개포로15길 27,서울특별시 강남구 개포동,1239,6,강남빌라나동,52.75,22000,2.0,1988.0,-,-,매매,18.0,2006-11-02,417.061611,34.0,NaN,NaN,NaN,NaN,NaN,연립다세대
4,서울특별시 강남구 개포동 1175-2,논현로6길 36-8,서울특별시 강남구 개포동,1175,2,강남파크,49.43,17000,4.0,1993.0,-,-,매매,13.0,2006-07-19,343.920696,29.0,NaN,NaN,NaN,NaN,NaN,연립다세대


In [10]:
officetel_trade_df = data_prep('오피스텔', '매매')
officetel_lease_df = data_prep('오피스텔', '전월세')
officetel_df = pd.concat([officetel_trade_df, officetel_lease_df]).reset_index(drop=True)
del officetel_trade_df, officetel_lease_df
officetel_df['부동산유형'] = '오피스텔'
print(officetel_df.shape)
officetel_df.head()

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


0it [00:00, ?it/s]

0it [00:00, ?it/s]

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:2: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  


(353158, 23)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,거래유형,중개사소재지,거래구분,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,계약기간,계약구분,갱신요구권 사용,종전계약 보증금 (만원),종전계약 월세 (만원),부동산유형
0,서울특별시 강남구 개포동 13-3,개포로 623,서울특별시 강남구 개포동,13,3,대청타워,32.44,6400,8,1997.0,-,-,매매,9.0,2006-01-10,197.287300,25.0,NaN,NaN,NaN,NaN,NaN,오피스텔
1,서울특별시 강남구 개포동 13-3,개포로 623,서울특별시 강남구 개포동,13,3,대청타워,32.44,7100,7,1997.0,-,-,매매,9.0,2006-01-10,218.865598,25.0,NaN,NaN,NaN,NaN,NaN,오피스텔
2,서울특별시 강남구 개포동 13-3,개포로 623,서울특별시 강남구 개포동,13,3,대청타워,32.44,7300,11,1997.0,-,-,매매,9.0,2006-01-11,225.030826,25.0,NaN,NaN,NaN,NaN,NaN,오피스텔
3,서울특별시 강남구 개포동 13-3,개포로 623,서울특별시 강남구 개포동,13,3,대청타워,31.91,7500,10,1997.0,-,-,매매,9.0,2006-01-18,235.036039,25.0,NaN,NaN,NaN,NaN,NaN,오피스텔
4,서울특별시 강남구 개포동 13-3,개포로 623,서울특별시 강남구 개포동,13,3,대청타워,32.44,7400,15,1997.0,-,-,매매,9.0,2006-01-19,228.113440,25.0,NaN,NaN,NaN,NaN,NaN,오피스텔


In [11]:
house_df = pd.concat([apart_df, yunrip_df, officetel_df]).sort_values(['시군구', '본번', '부번', '건축년도', '전용면적(㎡)', '계약날짜']).reset_index(drop=True)
house_df['건축년도'] = house_df['건축년도'].astype('str')
del apart_df, yunrip_df, officetel_df
print(house_df.shape)
house_df.head()

(4154272, 23)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,거래유형,중개사소재지,거래구분,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,계약기간,계약구분,갱신요구권 사용,종전계약 보증금 (만원),종전계약 월세 (만원),부동산유형
0,서울특별시 강남구 개포동 12-0,개포로109길 9,서울특별시 강남구 개포동,12.0,0.0,성원대치2단지아파트,33.18,15900,1.0,1992.0,-,-,매매,14.0,2006-01-03,479.204340,30.0,NaN,NaN,NaN,NaN,NaN,아파트
1,서울특별시 강남구 개포동 12-0,개포로109길 9,서울특별시 강남구 개포동,12.0,0.0,성원대치2단지아파트,33.18,18400,5.0,1992.0,-,-,매매,14.0,2006-01-17,554.550934,30.0,NaN,NaN,NaN,NaN,NaN,아파트
2,서울특별시 강남구 개포동 12-0,개포로109길 9,서울특별시 강남구 개포동,12.0,0.0,성원대치2단지아파트,33.18,18300,3.0,1992.0,-,-,매매,14.0,2006-02-20,551.537071,30.0,NaN,NaN,NaN,NaN,NaN,아파트
3,서울특별시 강남구 개포동 12-0,개포로109길 9,서울특별시 강남구 개포동,12.0,0.0,성원대치2단지아파트,33.18,17200,15.0,1992.0,-,-,매매,14.0,2006-03-13,518.384569,30.0,NaN,NaN,NaN,NaN,NaN,아파트
4,서울특별시 강남구 개포동 12-0,개포로109길 9,서울특별시 강남구 개포동,12.0,0.0,성원대치2단지아파트,33.18,17600,15.0,1992.0,-,-,매매,14.0,2006-03-14,530.440024,30.0,NaN,NaN,NaN,NaN,NaN,아파트


In [12]:
house_df = house_df.drop(columns=['계약기간', '계약구분', '갱신요구권 사용', '종전계약 보증금 (만원)', '종전계약 월세 (만원)'])
print(house_df.shape)
house_df.head()

(4154272, 18)


,지번주소,도로명,시군구,본번,부번,건물(단지)명,전용면적(㎡),거래금액(만원),층,건축년도,거래유형,중개사소재지,거래구분,계약날짜기준_건물연식,계약날짜,단가(만원/㎡),건물연식,부동산유형
0,서울특별시 강남구 개포동 12-0,개포로109길 9,서울특별시 강남구 개포동,12.0,0.0,성원대치2단지아파트,33.18,15900,1.0,1992.0,-,-,매매,14.0,2006-01-03,479.204340,30.0,아파트
1,서울특별시 강남구 개포동 12-0,개포로109길 9,서울특별시 강남구 개포동,12.0,0.0,성원대치2단지아파트,33.18,18400,5.0,1992.0,-,-,매매,14.0,2006-01-17,554.550934,30.0,아파트
2,서울특별시 강남구 개포동 12-0,개포로109길 9,서울특별시 강남구 개포동,12.0,0.0,성원대치2단지아파트,33.18,18300,3.0,1992.0,-,-,매매,14.0,2006-02-20,551.537071,30.0,아파트
3,서울특별시 강남구 개포동 12-0,개포로109길 9,서울특별시 강남구 개포동,12.0,0.0,성원대치2단지아파트,33.18,17200,15.0,1992.0,-,-,매매,14.0,2006-03-13,518.384569,30.0,아파트
4,서울특별시 강남구 개포동 12-0,개포로109길 9,서울특별시 강남구 개포동,12.0,0.0,성원대치2단지아파트,33.18,17600,15.0,1992.0,-,-,매매,14.0,2006-03-14,530.440024,30.0,아파트


In [13]:
house_df['구'] = house_df['지번주소'].apply(lambda x: x.split(' ')[1])
house_df['동'] = house_df['지번주소'].apply(lambda x: x.split(' ')[2])
#house_merge_df['구'] = house_merge_df['지번주소'].apply(lambda x: x.split(' ')[1])
#house_merge_df['동'] = house_merge_df['지번주소'].apply(lambda x: x.split(' ')[2])

In [14]:
house_df['일괄계약'] = 'N'

In [15]:
house_df['건축년도'].isna().sum()

0

In [16]:
house_df['건축년도'].dtype

dtype('O')

In [17]:
(house_df['건축년도'] == 'nan').sum()

7513

In [18]:
print(house_df.shape)
house_df = house_df[house_df['건축년도'] != 'nan'].reset_index(drop=True)
print(house_df.shape)

(4154272, 21)
(4146759, 21)


In [19]:
house_df['건축년도'] = house_df['건축년도'].apply(float).apply(int).astype('Int64')

In [20]:
house_df['지번주소_건축년도'] = house_df['지번주소'] + ' ' + house_df['건축년도'].astype('str')

In [21]:
def identify_bulk_contract(df):
    df = df.copy()
    
    for addr in df['지번주소'].unique():
        addr_df = df[df['지번주소'] == addr]
        
        for btyear in addr_df['건축년도'].unique():
            btyear_df = addr_df[addr_df['건축년도'] == btyear]
        
            for contract_date in btyear_df['계약날짜'].unique():
                contract_date_df = btyear_df[btyear_df['계약날짜'] == contract_date]

                if contract_date_df.shape[0] >= 5:
                    df.loc[contract_date_df.index, '일괄계약'] = 'Y'
    
    return df

In [22]:
def get_sales_idx(x):
    # x is a pandas timestamp
    
    df = sales_idx_df.copy()
    
    if pd.isna(x) == True:
        return x
    else:
        year = str(x.year)
        month = str(x.month)
        if len(month) == 1:
            month = '0'+month
        
        try:
            idx = df[year+'년 '+month+'월_지수'].iloc[0]
            return idx
        
        except:
            return '지수없음'

In [23]:
def get_index_applied_prices(df):
    df = df.copy()
    
    last_idx = sales_idx_df.iloc[0,-2]
    
    df['계약날짜_실거래지수'] = df['계약날짜'].apply(get_sales_idx)
    
    df_idx_positive = df[df['계약날짜_실거래지수'] != '지수없음']
    
    df['지수적용단가'] = df['단가(만원/㎡)']
    
    df_idx_positive['지수적용단가'] = df_idx_positive['단가(만원/㎡)'] / df_idx_positive['계약날짜_실거래지수'] * last_idx
    
    df.loc[df_idx_positive.index, '지수적용단가'] = df_idx_positive['지수적용단가']
    
    df['지수적용단가'] = df['지수적용단가'].astype('float')
    
    df['지수적용날짜'] = '지수없음'
    
    last_date = sales_idx_df.columns[-1].split('_')[0]
    df.loc[df_idx_positive.index, '지수적용날짜'] = last_date
    
    return df

In [24]:
sales_idx_basedir = './한국부동산원/실거래가격지수/'
filenames = [f for f in os.listdir(sales_idx_basedir) if '연립다세대' in f]

sales_idx_df = pd.read_excel(sales_idx_basedir + filenames[0], header=[10, 11])

sales_idx_cols_list = []
for i in range(sales_idx_df.shape[1]):
    if i == 0:
        sales_idx_cols_list.append(sales_idx_df.columns[i][0])
    else:
        sales_idx_cols_list.append(sales_idx_df.columns[i][0] + '_' + sales_idx_df.columns[i][1])

sales_idx_df.columns = sales_idx_cols_list

In [25]:
print(sales_idx_df.shape)
sales_idx_df.head()

(1, 381)


,지 역,2006년 01월_지수,2006년 01월_변동률,2006년 02월_지수,2006년 02월_변동률,2006년 03월_지수,2006년 03월_변동률,2006년 04월_지수,2006년 04월_변동률,2006년 05월_지수,2006년 05월_변동률,2006년 06월_지수,2006년 06월_변동률,2006년 07월_지수,2006년 07월_변동률,2006년 08월_지수,2006년 08월_변동률,2006년 09월_지수,2006년 09월_변동률,2006년 10월_지수,2006년 10월_변동률,2006년 11월_지수,2006년 11월_변동률,2006년 12월_지수,2006년 12월_변동률,2007년 01월_지수,2007년 01월_변동률,2007년 02월_지수,2007년 02월_변동률,2007년 03월_지수,2007년 03월_변동률,2007년 04월_지수,2007년 04월_변동률,2007년 05월_지수,2007년 05월_변동률,2007년 06월_지수,2007년 06월_변동률,2007년 07월_지수,2007년 07월_변동률,2007년 08월_지수,2007년 08월_변동률,2007년 09월_지수,2007년 09월_변동률,2007년 10월_지수,2007년 10월_변동률,2007년 11월_지수,2007년 11월_변동률,2007년 12월_지수,2007년 12월_변동률,2008년 01월_지수,2008년 01월_변동률,2008년 02월_지수,2008년 02월_변동률,2008년 03월_지수,2008년 03월_변동률,2008년 04월_지수,2008년 04월_변동률,2008년 05월_지수,2008년 05월_변동률,2008년 06월_지수,2008년 06월_변동률,2008년 07월_지수,2008년 07월_변동률,2008년 08월_지수,2008년 08월_변동률,2008년 09월_지수,2008년 09월_변동률,2008년 10월_지수,2008년 10월_변동률,2008년 11월_지수,2008년 11월_변동률,2008년 12월_지수,2008년 12월_변동률,2009년 01월_지수,2009년 01월_변동률,2009년 02월_지수,2009년 02월_변동률,2009년 03월_지수,2009년 03월_변동률,2009년 04월_지수,2009년 04월_변동률,2009년 05월_지수,2009년 05월_변동률,2009년 06월_지수,2009년 06월_변동률,2009년 07월_지수,2009년 07월_변동률,2009년 08월_지수,2009년 08월_변동률,2009년 09월_지수,2009년 09월_변동률,2009년 10월_지수,2009년 10월_변동률,2009년 11월_지수,2009년 11월_변동률,2009년 12월_지수,2009년 12월_변동률,2010년 01월_지수,2010년 01월_변동률,2010년 02월_지수,2010년 02월_변동률,2010년 03월_지수,2010년 03월_변동률,2010년 04월_지수,2010년 04월_변동률,2010년 05월_지수,2010년 05월_변동률,2010년 06월_지수,2010년 06월_변동률,2010년 07월_지수,2010년 07월_변동률,2010년 08월_지수,2010년 08월_변동률,2010년 09월_지수,2010년 09월_변동률,2010년 10월_지수,2010년 10월_변동률,2010년 11월_지수,2010년 11월_변동률,2010년 12월_지수,2010년 12월_변동률,2011년 01월_지수,2011년 01월_변동률,2011년 02월_지수,2011년 02월_변동률,2011년 03월_지수,2011년 03월_변동률,2011년 04월_지수,2011년 04월_변동률,2011년 05월_지수,2011년 05월_변동률,2011년 06월_지수,2011년 06월_변동률,2011년 07월_지수,2011년 07월_변동률,2011년 08월_지수,2011년 08월_변동률,2011년 09월_지수,2011년 09월_변동률,2011년 10월_지수,2011년 10월_변동률,2011년 11월_지수,2011년 11월_변동률,2011년 12월_지수,2011년 12월_변동률,2012년 01월_지수,2012년 01월_변동률,2012년 02월_지수,2012년 02월_변동률,2012년 03월_지수,2012년 03월_변동률,2012년 04월_지수,2012년 04월_변동률,2012년 05월_지수,2012년 05월_변동률,2012년 06월_지수,2012년 06월_변동률,2012년 07월_지수,2012년 07월_변동률,2012년 08월_지수,2012년 08월_변동률,2012년 09월_지수,2012년 09월_변동률,2012년 10월_지수,2012년 10월_변동률,2012년 11월_지수,2012년 11월_변동률,2012년 12월_지수,2012년 12월_변동률,2013년 01월_지수,2013년 01월_변동률,2013년 02월_지수,2013년 02월_변동률,2013년 03월_지수,2013년 03월_변동률,2013년 04월_지수,2013년 04월_변동률,2013년 05월_지수,2013년 05월_변동률,2013년 06월_지수,2013년 06월_변동률,2013년 07월_지수,2013년 07월_변동률,2013년 08월_지수,2013년 08월_변동률,2013년 09월_지수,2013년 09월_변동률,2013년 10월_지수,2013년 10월_변동률,2013년 11월_지수,2013년 11월_변동률,2013년 12월_지수,2013년 12월_변동률,2014년 01월_지수,2014년 01월_변동률,2014년 02월_지수,2014년 02월_변동률,2014년 03월_지수,2014년 03월_변동률,2014년 04월_지수,2014년 04월_변동률,2014년 05월_지수,2014년 05월_변동률,2014년 06월_지수,2014년 06월_변동률,2014년 07월_지수,2014년 07월_변동률,2014년 08월_지수,2014년 08월_변동률,2014년 09월_지수,2014년 09월_변동률,2014년 10월_지수,2014년 10월_변동률,2014년 11월_지수,2014년 11월_변동률,2014년 12월_지수,2014년 12월_변동률,2015년 01월_지수,2015년 01월_변동률,2015년 02월_지수,2015년 02월_변동률,2015년 03월_지수,2015년 03월_변동률,2015년 04월_지수,2015년 04월_변동률,2015년 05월_지수,2015년 05월_변동률,2015년 06월_지수,2015년 06월_변동률,2015년 07월_지수,2015년 07월_변동률,2015년 08월_지수,2015년 08월_변동률,2015년 09월_지수,2015년 09월_변동률,2015년 10월_지수,2015년 10월_변동률,2015년 11월_지수,2015년 11월_변동률,2015년 12월_지수,2015년 12월_변동률,2016년 01월_지수,2016년 01월_변동률,2016년 02월_지수,2016년 02월_변동률,2016년 03월_지수,2016년 03월_변동률,2016년 04월_지수,2016년 04월_변동률,2016년 05월_지수,2016년 05월_변동률,2016년 06월_지수,2016년 06월_변동률,2016년 07월_지수,2016년 07월_변동률,2016년 08월_지수,2016년 08월_변동률,2016년 09월_지수,2016년 09월_변동률,2016년 10월_지수,2016년 10월_변동률,2016년 11월_지수,2016년 11월_변동률,2016년 12월_지수,2016년 12월_변동률,2017년 01월_지수,2017년 01월_변동률,2017년 02월_지수,2017년 02월_변동률,2017년 03월_지수,2017년 03월_변동률,2017년 04월_지수,2017년 04월_변동률,2017년 05월_지수,2017년 05월_변동률,2017년 06월_지수,2017년 06월_변동률,2017년 07월_지수,2017년 07월_변동률,2017년 08월_지수,2017년 08월_변동률,2017년 09월_지수,2017년 09월_변동률,2017년 10월_지수,2017년 10월_변동률,2017년 11월_지수,2017년 11월_변동률,2017년 12월_지수,2017년 12월_변동률,2018년 01월_지수,2018년 01월_변동률,2018년 02월_지수,2018년 02월_변동률,2018년 03월_지수,2018년 03월_변동률,2018년 04월_지수,2018년 04월_변동률

In [26]:
sales_idx_df_original = sales_idx_df.copy()

In [27]:
del sales_idx_df

In [28]:
try:
    os.makedirs('./국토교통부_실거래가_공개시스템/집값분석/연습용자료/')
except:
    pass

In [29]:
def month_length_mod(x):
    if len(x) == 1:
        x = '0' + x
    
    return x

In [30]:
def remove_numbers(x):
    for i in range(10):
        i = str(i)
        x = x.replace(i, '')
    
    return x

In [31]:
count = 0

for addr in tqdm.tqdm_notebook(target_df['지번주소_건축년도'].unique()):    
    addr_df = target_df[target_df['지번주소_건축년도'] == addr]
    
    built_year = int(addr.split(' ')[-1])
    
    cols_to_include = [
        '지번주소', '건물(단지)명', '건물연식', '층', '거래구분', '전용면적(㎡)', '계약날짜', '거래금액(만원)', '단가(만원/㎡)',
        '부동산유형', '일괄계약', '지수적용날짜', '지수적용단가'
    ]
    
    first_df_rows = np.nan
    second_df_rows = np.nan
    third_df_rows = np.nan
    fourth_df_rows = np.nan
    
    cols_to_include_first = cols_to_include.copy()
    cols_to_include_second = cols_to_include.copy()
    cols_to_include_third = cols_to_include.copy()
    cols_to_include_fourth = cols_to_include.copy()
    
    splitted = addr.split(' ')
    target_gu = splitted[0]
    target_dong = splitted[1].replace('동', '')
    target_dong = remove_numbers(target_dong)
    date_today = pd.to_datetime(addr_df['계약날짜'].iloc[0])
    date_today_record = str(date_today).split(' ')[0].replace('-', '')
    
    idx_year = str(int(date_today.year))
    idx_month = str(int(date_today.month))
    idx_month = month_length_mod(idx_month)
    
    idx_range_lim = '{}년 {}월_변동률'.format(idx_year, idx_month)
    
    sales_idx_df = sales_idx_df_original.loc[:0, :idx_range_lim]
    
    house_df['건물연식'] = built_year - house_df['건축년도']
    
    addr = '서울특별시 ' + addr
    print(addr)
    
    target_1_12_df = house_df[
        (house_df['지번주소'].str.contains(target_gu + ' ' + target_dong))
        &(house_df['지번주소_건축년도'] != addr)
        #&(house_df['동'] == target_dong)
        &(house_df['계약날짜'] <= date_today)
        &(house_df['계약날짜'] > (date_today - pd.DateOffset(months=6)))
        #&(house_df['건물연식'] <= target_building_age)
        #&(house_merge_df['용도지역명1'].str.contains(target_purpose))
    ]
    
    first_df_rows = target_1_12_df.shape[0]
    
    target_2_24_df = house_df[
        (house_df['지번주소'].str.contains(target_gu + ' ' + target_dong))
        &(house_df['지번주소_건축년도'] != addr)
        #&(house_df['동'] == target_dong)
        &(house_df['계약날짜'] <= date_today)
        &(house_df['계약날짜'] > (date_today - pd.DateOffset(months=12)))
        #&(house_df['건물연식'] <= target_building_age)
        #&(house_merge_df['용도지역명1'].str.contains(target_purpose))
    ]
    
    second_df_rows = target_2_24_df.shape[0]
    
    target_3_36_df = house_df[
        (house_df['지번주소'].str.contains(target_gu + ' ' + target_dong))
        &(house_df['지번주소_건축년도'] != addr)
        #&(house_df['동'] == target_dong)
        &(house_df['계약날짜'] <= date_today)
        &(house_df['계약날짜'] > (date_today - pd.DateOffset(months=24)))
        #&(house_df['건물연식'] <= target_building_age)
        #&(house_merge_df['용도지역명1'].str.contains(target_purpose))
    ]
    
    third_df_rows = target_3_36_df.shape[0]
    
    target_5_60_df = house_df[
        (house_df['지번주소'].str.contains(target_gu + ' ' + target_dong))
        &(house_df['지번주소_건축년도'] != addr)
        #&(house_df['동'] == target_dong)
        &(house_df['계약날짜'] <= date_today)
        &(house_df['계약날짜'] > (date_today - pd.DateOffset(months=36)))
        #&(house_df['건물연식'] <= target_building_age)
        #&(house_merge_df['용도지역명1'].str.contains(target_purpose))
    ]
    
    fourth_df_rows = target_5_60_df.shape[0]
    
    target_1_12_df = identify_bulk_contract(target_1_12_df)

    target_2_24_df = identify_bulk_contract(target_2_24_df)

    target_3_36_df = identify_bulk_contract(target_3_36_df)

    target_5_60_df = identify_bulk_contract(target_5_60_df)
    
    
    try:
        target_1_12_df = get_index_applied_prices(target_1_12_df)
    except:
        cols_to_include_first.remove('지수적용날짜')
        cols_to_include_first.remove('지수적용단가')
        pass

    try:
        target_2_24_df = get_index_applied_prices(target_2_24_df)
    except:
        cols_to_include_second.remove('지수적용날짜')
        cols_to_include_second.remove('지수적용단가')
        pass
    
    try:
        target_3_36_df = get_index_applied_prices(target_3_36_df)
    except:
        cols_to_include_third.remove('지수적용날짜')
        cols_to_include_third.remove('지수적용단가')
        pass
    
    try:
        target_5_60_df = get_index_applied_prices(target_5_60_df)
    except:
        cols_to_include_fourth.remove('지수적용날짜')
        cols_to_include_fourth.remove('지수적용단가')
        pass
    
    os.makedirs('./국토교통부_실거래가_공개시스템/집값분석/연습용자료/{}용/{}_{}_{}/'.format(addr, target_gu, target_dong, date_today_record))
    
    target_1_12_df = target_1_12_df.sort_values(['시군구', '본번', '부번', '건축년도', '부동산유형', '거래구분', '계약날짜', '전용면적(㎡)']).reset_index(drop=True)
    target_1_12_df1 = target_1_12_df[cols_to_include_first]
    target_1_12_df1.to_excel('./국토교통부_실거래가_공개시스템/집값분석/연습용자료/{}용/{}_{}_{}/실거래가_건물연식없음_계약최근6개월.xlsx'.format(addr, target_gu, target_dong, date_today_record), index=False)

    target_1_12_30_df1 = target_1_12_df1[target_1_12_df1['전용면적(㎡)'] <= 30]
    target_1_12_30_df1.to_excel('./국토교통부_실거래가_공개시스템/집값분석/연습용자료/{}용/{}_{}_{}/실거래가_건물연식없음_계약최근6개월_30제곱미터미만.xlsx'.format(addr, target_gu, target_dong, date_today_record), index=False)

    target_1_12_bdyr2_df1 = target_1_12_df1[target_1_12_df1['건물연식'] <= 2]
    target_1_12_bdyr2_df1.to_excel('./국토교통부_실거래가_공개시스템/집값분석/연습용자료/{}용/{}_{}_{}/실거래가_건물연식{}년_계약최근6개월.xlsx'.format(addr, target_gu, target_dong, date_today_record, 2), index=False)

    target_1_12_bdyr5_df1 = target_1_12_df1[target_1_12_df1['건물연식'] <= 5]
    target_1_12_bdyr5_df1.to_excel('./국토교통부_실거래가_공개시스템/집값분석/연습용자료/{}용/{}_{}_{}/실거래가_건물연식{}년_계약최근6개월.xlsx'.format(addr, target_gu, target_dong, date_today_record, 5), index=False)

    target_2_24_df = target_2_24_df.sort_values(['시군구', '본번', '부번', '건축년도', '부동산유형', '거래구분', '계약날짜', '전용면적(㎡)']).reset_index(drop=True)
    target_2_24_df1 = target_2_24_df[cols_to_include_second]
    target_2_24_df1.to_excel('./국토교통부_실거래가_공개시스템/집값분석/연습용자료/{}용/{}_{}_{}/실거래가_건물연식없음년_계약최근12개월.xlsx'.format(addr, target_gu, target_dong, date_today_record), index=False)

    target_2_24_30_df1 = target_2_24_df1[target_2_24_df1['전용면적(㎡)'] <= 30]
    target_2_24_30_df1.to_excel('./국토교통부_실거래가_공개시스템/집값분석/연습용자료/{}용/{}_{}_{}/실거래가_건물연식없음_계약최근12개월_30제곱미터미만.xlsx'.format(addr, target_gu, target_dong, date_today_record), index=False)

    target_2_24_bdyr2_df1 = target_2_24_df1[target_2_24_df1['건물연식'] <= 2]
    target_2_24_bdyr2_df1.to_excel('./국토교통부_실거래가_공개시스템/집값분석/연습용자료/{}용/{}_{}_{}/실거래가_건물연식{}년_계약최근12개월.xlsx'.format(addr, target_gu, target_dong, date_today_record, 2), index=False)

    target_2_24_bdyr5_df1 = target_2_24_df1[target_2_24_df1['건물연식'] <= 5]
    target_2_24_bdyr5_df1.to_excel('./국토교통부_실거래가_공개시스템/집값분석/연습용자료/{}용/{}_{}_{}/실거래가_건물연식{}년_계약최근12개월.xlsx'.format(addr, target_gu, target_dong, date_today_record, 5), index=False)

    target_3_36_df = target_3_36_df.sort_values(['시군구', '본번', '부번', '건축년도', '부동산유형', '거래구분', '계약날짜', '전용면적(㎡)']).reset_index(drop=True)
    target_3_36_df1 = target_3_36_df[cols_to_include_third]
    target_3_36_df1.to_excel('./국토교통부_실거래가_공개시스템/집값분석/연습용자료/{}용/{}_{}_{}/실거래가_건물연식없음_계약최근24개월.xlsx'.format(addr, target_gu, target_dong, date_today_record), index=False)

    target_3_36_30_df1 = target_3_36_df1[target_3_36_df1['전용면적(㎡)'] <= 30]
    target_3_36_30_df1.to_excel('./국토교통부_실거래가_공개시스템/집값분석/연습용자료/{}용/{}_{}_{}/실거래가_건물연식없음_계약최근24개월_30제곱미터미만.xlsx'.format(addr, target_gu, target_dong, date_today_record), index=False)

    target_3_36_bdyr2_df1 = target_3_36_df1[target_3_36_df1['건물연식'] <= 2]
    target_3_36_bdyr2_df1.to_excel('./국토교통부_실거래가_공개시스템/집값분석/연습용자료/{}용/{}_{}_{}/실거래가_건물연식{}년_계약최근24개월.xlsx'.format(addr, target_gu, target_dong, date_today_record, 2), index=False)

    target_3_36_bdyr5_df1 = target_3_36_df1[target_3_36_df1['건물연식'] <= 5]
    target_3_36_bdyr5_df1.to_excel('./국토교통부_실거래가_공개시스템/집값분석/연습용자료/{}용/{}_{}_{}/실거래가_건물연식{}년_계약최근24개월.xlsx'.format(addr, target_gu, target_dong, date_today_record, 5), index=False)

    target_5_60_df = target_5_60_df.sort_values(['시군구', '본번', '부번', '건축년도', '부동산유형', '거래구분', '계약날짜', '전용면적(㎡)']).reset_index(drop=True)
    target_5_60_df1 = target_5_60_df[cols_to_include_fourth]
    target_5_60_df1.to_excel('./국토교통부_실거래가_공개시스템/집값분석/연습용자료/{}용/{}_{}_{}/실거래가_건물연식없음_계약최근36개월.xlsx'.format(addr, target_gu, target_dong, date_today_record), index=False)

    target_5_60_30_df1 = target_5_60_df1[target_5_60_df1['전용면적(㎡)'] <= 30]
    target_5_60_30_df1.to_excel('./국토교통부_실거래가_공개시스템/집값분석/연습용자료/{}용/{}_{}_{}/실거래가_건물연식없음_계약최근36개월_30제곱미터미만.xlsx'.format(addr, target_gu, target_dong, date_today_record), index=False)

    target_5_60_bdyr2_df1 = target_5_60_df1[target_5_60_df1['건물연식'] <= 2]
    target_5_60_bdyr2_df1.to_excel('./국토교통부_실거래가_공개시스템/집값분석/연습용자료/{}용/{}_{}_{}/실거래가_건물연식{}년_계약최근36개월.xlsx'.format(addr, target_gu, target_dong, date_today_record, 2), index=False)

    target_5_60_bdyr5_df1 = target_5_60_df1[target_5_60_df1['건물연식'] <= 5]
    target_5_60_bdyr5_df1.to_excel('./국토교통부_실거래가_공개시스템/집값분석/연습용자료/{}용/{}_{}_{}/실거래가_건물연식{}년_계약최근36개월.xlsx'.format(addr, target_gu, target_dong, date_today_record, 5), index=False)

c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/848 [00:00<?, ?it/s]

서울특별시 강남구 개포동 1237-7 2021
서울특별시 강남구 개포동 1216-7 2020
서울특별시 강남구 개포동 1195-10 2020
서울특별시 강남구 개포동 1199-7 2019
서울특별시 강남구 개포동 1244-12 2018
서울특별시 강남구 역삼동 778-19 2021
서울특별시 강남구 역삼동 777-11 2021
서울특별시 강남구 역삼동 782-26 2020
서울특별시 강남구 역삼동 707-9 2020
서울특별시 강남구 역삼동 778-15 2020
서울특별시 강남구 역삼동 778-3 2020
서울특별시 강남구 역삼동 751-6 2019
서울특별시 강동구 고덕동 197-1 2021
서울특별시 강동구 고덕동 194-3 2021
서울특별시 강동구 고덕동 292-5 2020
서울특별시 강동구 고덕동 292-6 2020
서울특별시 강동구 길동 369-5 2021
서울특별시 강동구 길동 96-4 2020
서울특별시 강동구 길동 349-9 2020
서울특별시 강동구 길동 140-0 2020
서울특별시 강동구 길동 359-16 2020
서울특별시 강동구 길동 369-4 2020
서울특별시 강동구 길동 139-0 2019
서울특별시 강동구 길동 127-8 2018
서울특별시 강동구 명일동 350-1 2020
서울특별시 강동구 상일동 346-0 2018
서울특별시 강동구 성내동 436-2 2021
서울특별시 강동구 성내동 434-40 2021
서울특별시 강동구 성내동 404-13 2021
서울특별시 강동구 성내동 275-3 2021
서울특별시 강동구 성내동 529-2 2020
서울특별시 강동구 성내동 461-24 2020
서울특별시 강동구 성내동 127-13 2020
서울특별시 강동구 성내동 420-3 2020
서울특별시 강동구 성내동 461-20 2020
서울특별시 강동구 성내동 226-6 2016
서울특별시 강동구 성내동 461-3 2019
서울특별시 강동구 성내동 440-26 2019
서울특별시 강동구 성내동 440-5 2019
서울특별시 강동구 성내동 28

서울특별시 금천구 시흥동 927-21 2020
서울특별시 금천구 시흥동 924-18 2020
서울특별시 금천구 시흥동 825-24 2020
서울특별시 금천구 시흥동 790-8 2020
서울특별시 금천구 시흥동 230-50 2019
서울특별시 금천구 시흥동 960-10 2020
서울특별시 금천구 시흥동 237-45 2020
서울특별시 금천구 시흥동 220-161 2019
서울특별시 금천구 시흥동 239-23 2020
서울특별시 금천구 시흥동 827-17 2020
서울특별시 금천구 시흥동 946-17 2020
서울특별시 금천구 시흥동 863-16 2019
서울특별시 금천구 시흥동 857-12 2020
서울특별시 금천구 시흥동 790-19 2019
서울특별시 금천구 시흥동 230-15 2019
서울특별시 금천구 시흥동 204-11 2019
서울특별시 금천구 시흥동 813-43 2019
서울특별시 금천구 시흥동 925-18 2019
서울특별시 금천구 시흥동 935-83 2019
서울특별시 금천구 시흥동 220-49 2019
서울특별시 금천구 시흥동 848-26 2019
서울특별시 금천구 시흥동 849-32 2019
서울특별시 금천구 시흥동 849-33 2019
서울특별시 금천구 시흥동 864-23 2019
서울특별시 금천구 시흥동 239-12 2019
서울특별시 금천구 시흥동 919-78 2019
서울특별시 금천구 시흥동 813-33 2019
서울특별시 금천구 시흥동 220-1 2019
서울특별시 금천구 시흥동 919-88 2019
서울특별시 금천구 시흥동 827-29 2019
서울특별시 금천구 시흥동 239-25 2019
서울특별시 금천구 시흥동 819-18 2019
서울특별시 금천구 시흥동 230-121 2019
서울특별시 금천구 시흥동 815-9 2019
서울특별시 금천구 시흥동 220-53 2019
서울특별시 금천구 시흥동 811-7 2019
서울특별시 금천구 시흥동 897-2 2018
서울특별시 금천구 시흥동 212-6 2018
서울특별시 금천구 시흥동 22

서울특별시 송파구 가락동 13-0 2020
서울특별시 송파구 가락동 198-5 2020
서울특별시 송파구 가락동 11-18 2020
서울특별시 송파구 가락동 182-0 2019
서울특별시 송파구 가락동 182-1 2019
서울특별시 송파구 가락동 47-10 2012
서울특별시 송파구 거여동 251-22 2019
서울특별시 송파구 마천동 14-4 2020
서울특별시 송파구 문정동 35-11 2020
서울특별시 송파구 문정동 35-13 2020
서울특별시 송파구 방이동 111-3 2021
서울특별시 송파구 방이동 99-5 2021
서울특별시 송파구 방이동 211-11 2021
서울특별시 송파구 방이동 100-16 2020
서울특별시 송파구 방이동 151-23 2020
서울특별시 송파구 방이동 118-13 2020
서울특별시 송파구 방이동 161-17 2020
서울특별시 송파구 삼전동 117-15 2021
서울특별시 송파구 삼전동 170-14 2021
서울특별시 송파구 삼전동 171-21 2021
서울특별시 송파구 삼전동 171-6 2021
서울특별시 송파구 삼전동 64-6 2021
서울특별시 송파구 삼전동 171-19 2020
서울특별시 송파구 삼전동 171-20 2020
서울특별시 송파구 삼전동 61-7 2020
서울특별시 송파구 석촌동 165-8 2021
서울특별시 송파구 석촌동 245-19 2021
서울특별시 송파구 석촌동 54-19 2021
서울특별시 송파구 석촌동 164-8 2021
서울특별시 송파구 석촌동 252-5 2020
서울특별시 송파구 송파동 102-26 2021
서울특별시 송파구 송파동 29-5 2021
서울특별시 송파구 송파동 102-16 2021
서울특별시 송파구 송파동 180-3 2020
서울특별시 송파구 송파동 118-13 2021
서울특별시 송파구 송파동 117-31 2020
서울특별시 송파구 송파동 101-20 2020
서울특별시 송파구 송파동 145-5 2020
서울특별시 송파구 송파동 11-4 2020
서울특별시 송파구 송파동 1